In [1]:
import os

In [2]:
%pwd

'e:\\projects\\end_to_end_pneomonia_detection_deep_learning_project_using_MLFLOW_DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\projects\\end_to_end_pneomonia_detection_deep_learning_project_using_MLFLOW_DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "pneumonia-detection-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-02-17 17:16:13,759: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-17 17:16:13,763: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-17 17:16:13,765: INFO: common: created directory at: artifacts]
[2024-02-17 17:16:13,766: INFO: common: created directory at: artifacts\training]


Found 1170 images belonging to 2 classes.
Found 4686 images belonging to 2 classes.
Epoch 1/15
292/292 [==============================] - 483s 2s/step - loss: 5.5933 - accuracy: 0.7272 - val_loss: 1.0136 - val_accuracy: 0.8570
Epoch 2/15
292/292 [==============================] - 498s 2s/step - loss: 2.0622 - accuracy: 0.8287 - val_loss: 0.6731 - val_accuracy: 0.8870
Epoch 3/15
292/292 [==============================] - 466s 2s/step - loss: 1.6526 - accuracy: 0.8443 - val_loss: 1.0721 - val_accuracy: 0.8596
Epoch 4/15
292/292 [==============================] - 491s 2s/step - loss: 1.7638 - accuracy: 0.8565 - val_loss: 3.2873 - val_accuracy: 0.7277
Epoch 5/15
292/292 [==============================] - 468s 2s/step - loss: 1.3552 - accuracy: 0.8724 - val_loss: 0.8202 - val_accuracy: 0.8861
Epoch 6/15
292/292 [==============================] - 446s 2s/step - loss: 1.4888 - accuracy: 0.8687 - val_loss: 0.4730 - val_accuracy: 0.9195
Epoch 7/15
292/292 [==============================] - 416s